In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

import spykshrk.realtime.simulator.nspike_data as nspike_data

#pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 10)
#pd.set_option('display.width', 180)


idx = pd.IndexSlice

In [2]:
# Load merged rec HDF store based on config

config_file = '/opt/data36/daliu/realtime/spykshrk/ripple_dec/bond.config.json'
config = json.load(open(config_file, 'r'))

hdf_file = os.path.join(config['files']['output_dir'],
                        '{}.rec_merged.h5'.format(config['files']['prefix']))

store = pd.HDFStore(hdf_file, mode='r')
spike_decode = store['rec_3']
stim_lockout = store['rec_11']

In [3]:
stim_lockout_ranges = stim_lockout.pivot(index='lockout_num',columns='lockout_state', values='timestamp')
stim_lockout_ranges = stim_lockout_ranges.reindex(columns=[1,0])


In [4]:
# Get table with decode for each spike and generate decode bin mask

dec_bin_size = 750     # Decode bin size in samples (usually 30kHz)

dec_bins = np.floor((spike_decode['timestamp'] - spike_decode['timestamp'][0])/dec_bin_size).astype('int')
spike_decode['dec_bin'] = dec_bins

In [5]:
# Loop through each bin and generate normalized posterior estimate of location

pos_bin_delta = ((config['encoder']['position']['upper'] - config['encoder']['position']['lower']) / 
                 config['encoder']['position']['bins'])
pos_num_bins = config['encoder']['position']['bins']

dec_bin_ids = np.unique(dec_bins)
dec_est = np.zeros([dec_bin_ids[-1]+1, pos_num_bins])
dec_est_bin_time = np.floor(spike_decode['timestamp'][0] / dec_bin_size) * dec_bin_size

start_bin_time = np.floor(spike_decode['timestamp'][0] / dec_bin_size) * dec_bin_size
dec_bin_times = np.arange(start_bin_time, start_bin_time + dec_bin_size * len(dec_est), dec_bin_size)


In [11]:
%%time

groups = spike_decode.groupby('dec_bin')

for bin_ii, spikes_in_bin in groups:
    dec_in_bin = np.ones(pos_num_bins)
    for dec in spikes_in_bin.loc[:, 'x0':'x{:d}'.format(pos_num_bins-1)].values:
        dec_in_bin *= dec               
        dec_in_bin = dec_in_bin / (np.sum(dec_in_bin) * pos_bin_delta)


    dec_est[bin_id, :] = dec_in_bin

In [17]:
spikes_in_bin.values[:,4:154]